# **Imports**

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import mutual_info_score

# **Data loading and preprocessing**
The data should be in a CSV or XLSX format with genes (Geneid) as rows and BAM replicates as columns.

In [ ]:
# Mount Google Drive and load the data file

# Mount Google Drive
drive.mount("/content/drive", force_remount=True)

# Define the path to the data file
root_dir = "/content/drive/MyDrive/"
base_dir = os.path.join(root_dir, "YOUR_PATH/")
data_file = 'your_file.xlsx'  # For example data.xlsx

# Check if the file is CSV or XLSX and load it into a pandas DataFrame
file_path = os.path.join(base_dir, data_file)
if data_file.endswith('.csv'):
    data_df = pd.read_csv(file_path, delimiter=",", index_col=0, header=0)
elif data_file.endswith('.xlsx'):
    data_df = pd.read_excel(file_path, index_col=0, header=0)
else:
    raise ValueError("Unsupported file format. Please provide a CSV or XLSX file.")

# Display the DataFrame
data_df

**Removing Zero Values from the Dataset**

In [ ]:
def remove_zeros(df):
    """
    Remove rows with any zeros (except the header).
    Save the resulting DataFrame to a new Excel file.
    """
    # Remove rows containing any zeros (excluding the header)
    df = df[~(df.iloc[:, 1:] == 0).any(axis=1)]

    # Save the resulting DataFrame to a new Excel file
    df.to_excel(os.path.join(base_dir, 'data_without_zeros.xlsx'))

    return df

# Remove rows with zeros and display the resulting DataFrame
data_df_without_zeros = remove_zeros(data_df)
data_df_without_zeros

# **Detection of Regulon Structures in the Entire Dataset**

In [ ]:
# Set the random seed for reproducibility
np.random.seed(0)

# Transpose the data for correlation coefficient and mutual information calculation
gene_expression_data_T = data_df_without_zeros.T

# Calculate the correlation matrix using Spearman method
corr_matrix = gene_expression_data_T.corr(method='spearman')

# Create an empty DataFrame for mutual information
mi_matrix = pd.DataFrame(index=gene_expression_data_T.columns, columns=gene_expression_data_T.columns, data=0.0)

# Calculate mutual information between columns
for gen1 in gene_expression_data_T.columns:
    for gen2 in gene_expression_data_T.columns:
        if gen1 != gen2:
            mi_matrix.loc[gen1, gen2] = mutual_info_regression(gene_expression_data_T[[gen1]], gene_expression_data_T[gen2])[0]

# Copy the lower half of the matrix to the upper half using numpy
mi_matrix.values[np.triu_indices_from(mi_matrix, 1)] = mi_matrix.values.T[np.triu_indices_from(mi_matrix, 1)]

# Set diagonal values to the maximum value in the matrix
max_mi_value = mi_matrix.max().max()  # The highest value outside the diagonal
np.fill_diagonal(mi_matrix.values, max_mi_value)

# Normalize the correlation matrix to the range 0-1
corr_matrix = (corr_matrix - corr_matrix.min().min()) / (corr_matrix.max().max() - corr_matrix.min().min())

# Normalize the mutual information matrix to the range 0-1
mi_matrix = (mi_matrix - mi_matrix.min().min()) / (mi_matrix.max().max() - mi_matrix.min().min())

# Create a result matrix with threshold values
result_matrix = pd.DataFrame(np.zeros_like(corr_matrix), index=corr_matrix.index, columns=corr_matrix.columns)

# Define thresholds from lowest to highest
thresholds = [0.2, 0.4, 0.6, 0.8, 1]

# Create an intersection matrix based on defined thresholds
for threshold in thresholds:
    condition = (corr_matrix >= threshold) & (mi_matrix >= threshold)
    result_matrix[condition] = threshold

# Save the result matrix to an Excel file
result_matrix.to_excel(os.path.join(base_dir, 'result_matrix.xlsx'))

# Visualize the result matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(result_matrix, cmap='viridis', annot=True)
plt.title('Heatmap of Potential Regulon Structures')
plt.xlabel('Genes')
plt.ylabel('Genes')
plt.show()

# UPGMA visualization using hierarchical clustering
linkage_matrix = linkage(1 - result_matrix, method='average')

plt.figure(figsize=(8, 8))
dendrogram(linkage_matrix, labels=mi_matrix.index, orientation='left', distance_sort='descending')
plt.title('UPGMA Dendrogram')
plt.xlabel('Genes')
plt.ylabel('Distance')
plt.show()

# Create a binary matrix for values above the 0.8 threshold (but not 1)
binary_matrix = pd.DataFrame(np.zeros_like(corr_matrix), index=corr_matrix.index, columns=corr_matrix.columns)
condition_08 = (result_matrix >= 0.8) & (result_matrix < 1)
binary_matrix[condition_08] = 1

# Save the binary matrix to an Excel file
binary_matrix.to_excel(os.path.join(base_dir, 'binary_matrix.xlsx'))

# Visualize the binary matrix
plt.figure(figsize=(8, 8))
sns.heatmap(binary_matrix, cmap='binary', annot=True, cbar=False)
plt.title('Binary Image of Predicted Potential Regulon Structures')
plt.xlabel('Genes')
plt.ylabel('Genes')
plt.show()

WHY SETTING DIAGONAL VALUES IN MI_MATRIX IS NECESSARY?

The mutual_info_regression function from the scikit-learn library is designed to estimate mutual information between two continuous variables. When applied to the same variables, the mutual information should theoretically be maximal. However, in practice, it may not always be consistent due to the way the function is implemented and how the continuous values are handled. Setting the diagonal values to the maximum value observed ensures that self-information is correctly represented as the highest value in the matrix.

# **Detection of Regulon Structures for a Specific Gene (Geneid)**

In [ ]:
# Set the random seed for reproducibility
np.random.seed(0)

# Define the target gene for analysis
target_gene = "YOUR_GENE"  # For example AT3G54230

# Check if the target gene exists in the data
if target_gene not in data_df_without_zeros.index:
    raise ValueError(f"Gene {target_gene} not found in the data.")

# Transpose the data for correlation coefficient and mutual information calculation
gene_expression_data_T = data_df_without_zeros.T

# Calculate the correlation matrix for the target gene against all other genes using Spearman method
corr_matrix = pd.DataFrame(index=gene_expression_data_T.columns, columns=[target_gene])

for gene in gene_expression_data_T.columns:
    corr_matrix.loc[gene, target_gene] = gene_expression_data_T[target_gene].corr(gene_expression_data_T[gene], method='spearman')

# Calculate the mutual information matrix for the target gene against all other genes
mi_matrix = pd.DataFrame(index=gene_expression_data_T.columns, columns=[target_gene])

for gene in gene_expression_data_T.columns:
    mi_value = mutual_info_regression(gene_expression_data_T[[target_gene]], gene_expression_data_T[gene])[0]
    mi_matrix.loc[gene, target_gene] = mi_value

# Normalize the correlation matrix to the range 0-1
corr_matrix = corr_matrix.astype(float)  # Ensure the matrix is of float type for normalization
corr_matrix = (corr_matrix - corr_matrix.min().min()) / (corr_matrix.max().max() - corr_matrix.min().min())

# Normalize the mutual information matrix to the range 0-1
mi_matrix = mi_matrix.astype(float)  # Ensure the matrix is of float type for normalization
mi_matrix = (mi_matrix - mi_matrix.min().min()) / (mi_matrix.max().max() - mi_matrix.min().min())

# Set the threshold for determining significant correlations and mutual information
threshold = 0.9

# Create a binary intersection matrix for values above the threshold of 0.9
result_matrix = pd.DataFrame(np.zeros_like(corr_matrix), index=corr_matrix.index, columns=corr_matrix.columns)
condition = (corr_matrix >= threshold) & (mi_matrix >= threshold)
result_matrix[condition] = 1

# Remove the target gene from the result matrix
result_matrix_no_target = result_matrix.drop(target_gene)

# Save the result binary matrix to an Excel file
result_matrix_no_target.to_excel(os.path.join(base_dir, 'result_matrix_no_target.xlsx'))

# Print the binary matrix as a DataFrame
print("Binary matrix for common regulation of gene expression with the specified gene:\n")
print(result_matrix_no_target)

# Count the number of ones in the binary matrix
total_number_of_ones = int(result_matrix_no_target.values.sum())

print(f"Number of coordinately regulated genes with {target_gene}: {total_number_of_ones}")